# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [54]:
import re
import Bio.SeqIO #these two lines of code get the necessary packages

def reverse_complement(seq, unk_partner = 'N'): #Defining a function to create the reverse transcript of our input sequence
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'} #defining the base partners
    rseq = ''#defining the "for" loop that finds a reverse transcription partner and adds, in reverse, to the growing reverse complement
    for a in seq: 
        if a in base_partner:
            pair = base_partner[a]
            rseq = pair + rseq
        else: #the else component of the for loop that accounts for non-defined partners covered by the above statement.
            rseq = unk_partner + rseq
    return rseq

seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq')) #Downloading the appropriate package of sequence reads using BioPython
seqreads_str = [] #Creating an empty list to throw sequence reads into
for seqrecord in seqreads: #For loop populating the liste we created above with individual seq reads
    seqreads_str.append(str(seqrecord.seq))

def read_HANA(seqread, bclen, downstream='AGGCGGCCGC'): #Creating a function using a regular expression to stratify sequences by a sequence, in this case the 27 nucleotide sequence before the shared common region
    reverse = reverse_complement(seqread) #Use our function defined above to get the reverse complement of a sequence
    HANA_pattern = re.compile(f"(?P<HANA>[ATCGN]{{{bclen}}})"+downstream) #A modified version of what we used in class, re-arranged to look at the sequence before the common region
    match = HANA_pattern.search(reverse) #A search function that looks for whatever pattern we define
    if match:
        HANA = match.group('HANA') #If there is a match, create a new grouping for that match
        return HANA
    else:
        return None
HANA_counts = {} #Create a dictionary for our groups defined above to be sorted into
n_invalid = 0 #Create a bucket for invalid reads

for seq in seqreads_str: # Apply the function defined above to the list of sequences to pull out the 27 nucleotide sequences before the shared common region
    HANA = read_HANA(seq, bclen = 27) #set the length probed to 27 nucleotides
    if HANA: # If there is a valid range of 27 nucleotides...
        if HANA in HANA_counts: #If the nucleotide sequence already exists in the dictionary, increase its count by one
            HANA_counts[HANA] += 1
        else: #If the nucleotide sequence does not exist in the dictionary, create it and set its count to 1
            HANA_counts[HANA] = 1
    else: # otherwise, add it to the count of invalid reads
        n_invalid += 1

print(f"The number of reads that mapped to NA were {HANA_counts['CACGATAGATAAATAATAGTGCACCAT']}") #print the number of NA & HA counts 
print(f"The number of reads that mapped to HA were {HANA_counts['CCGGATTTGCATATAATGATGCACCAT']}")

The number of reads that mapped to NA were 3973
The number of reads that mapped to HA were 5316


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [48]:
def read_barcodeNA(seqread, bclen, upstream='CACGATAGATAAATAATAGTGCACCATAGGCGGCCGC'): #Similar to above, this function creates a library, now looking at barcodes using the entire NA+common region as the requisite search feature
    reverseNA = reverse_complement(seqread) #take the reverse complement as we defined in Q1
    barcode_patternNA = re.compile(upstream + f"(?P<barcodeNA>[ATCGN]{{{bclen}}})$") #Use a regular expression to search for the barcode after the NA+coserved motif
    match = barcode_patternNA.search(reverseNA) #Search through all of our reverse complement
    if match: #If there is a match, create a new group for that match
        barcodeNA = match.group('barcodeNA')
        return barcodeNA
    else:
        return None

barcode_countsNA = {} #define a dictionary for NA barcode counts to be sorted into
n_invalidNA = 0 #set invalid reads to zero

for seq in seqreads_str: # iterate through all reads
    barcodeNA = read_barcodeNA(seq, bclen = 16) #Define seq as our input & the barcode length to 16 and apply our function
    if barcodeNA: # if there is a valid barcode, add it to the dictionary
        if barcodeNA in barcode_countsNA:
            barcode_countsNA[barcodeNA] += 1
        else:
            barcode_countsNA[barcodeNA] = 1
    else: # otherwise, add it to the count of invalid reads
        n_invalidNA += 1
barcode_countsNA

def read_barcodeHA(seqread, bclen, upstream='CCGGATTTGCATATAATGATGCACCATAGGCGGCCGC'): #Definine the same function as above, but with HA instead of NA
    reverseHA = reverse_complement(seqread) #Reverse read of seqread
    barcode_patternHA = re.compile(upstream + f"(?P<barcodeHA>[ATCGN]{{{bclen}}})$") #Defining a regular expression to search for barcodes using our defined motif
    match = barcode_patternHA.search(reverseHA) #use the regular expression to parse barcodes out of our reverse complements
    if match:
        barcodeHA = match.group('barcodeHA') #create groupings of the barcodes
        return barcodeHA
    else:
        return None

barcode_countsHA = {} #define a dictionary to dump barcodes into
n_invalidHA = 0 #set invalid reads to zero

for seq in seqreads_str: # iterate through all reads
    barcodeHA = read_barcodeHA(seq, bclen = 16) #apply newly defined function to seq, set barcode length to 16
    if barcodeHA: # if there is a valid barcode, add it to the dictionary
        if barcodeHA in barcode_countsHA:
            barcode_countsHA[barcodeHA] += 1
        else:
            barcode_countsHA[barcodeHA] = 1
    else: # otherwise, add it to the count of invalid reads
        n_invalidHA += 1
barcode_countsHA

print(f"The most common barcode for HA was {max(barcode_countsHA, key=barcode_countsHA.get)} with a total number of reads of {max(barcode_countsHA.values())}.") #print the top barcode ID and the number of reads for both HA and NA
print(f"The most common barcode for NA was {max(barcode_countsNA, key=barcode_countsNA.get)} with a total number of reads of {max(barcode_countsNA.values())}.")

The most common barcode for HA was CCCGACCCGACATTAA with a total number of reads of 155.
The most common barcode for NA was ACCAGTTCTCCCCGGG with a total number of reads of 152.


138